# [E-15] 아이유팬이 좋아할 만한 다른 아티스트 찾기

## 목차
0. import
1. 데이터 준비와 전처리
2. 분석해 봅시다.
3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
4. CSR matrix를 직접 만들어 봅시다.
5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.¶
6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
9. 회고

## 00. import

In [1]:
import numpy as np
import pandas as pd
import os

import scipy
import implicit
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 01. 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
# 사용할 컬럼
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# Data filtering 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


## 02. 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_top30 = movie_count.sort_values(ascending=False).head(30)

In [10]:
# 30개의 영화 누적 count수
for i, k in zip(movie_top30.index, movie_top30.values):
    print(movies[movies['movie_id']==i]['title'].values[0],k)

American Beauty (1999) 3211
Star Wars: Episode IV - A New Hope (1977) 2910
Star Wars: Episode V - The Empire Strikes Back (1980) 2885
Star Wars: Episode VI - Return of the Jedi (1983) 2716
Saving Private Ryan (1998) 2561
Terminator 2: Judgment Day (1991) 2509
Silence of the Lambs, The (1991) 2498
Raiders of the Lost Ark (1981) 2473
Back to the Future (1985) 2460
Matrix, The (1999) 2434
Jurassic Park (1993) 2413
Sixth Sense, The (1999) 2385
Fargo (1996) 2371
Braveheart (1995) 2314
Men in Black (1997) 2297
Schindler's List (1993) 2257
Princess Bride, The (1987) 2252
Shakespeare in Love (1998) 2213
L.A. Confidential (1997) 2210
Shawshank Redemption, The (1994) 2194
Godfather, The (1972) 2167
Groundhog Day (1993) 2121
E.T. the Extra-Terrestrial (1982) 2102
Being John Malkovich (1999) 2066
Ghostbusters (1984) 2051
Pulp Fiction (1994) 2030
Forrest Gump (1994) 2022
Terminator, The (1984) 2019
Toy Story (1995) 2000
Fugitive, The (1993) 1941


In [11]:
# 유저별 영화를 몇번 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [12]:
# 유저별 count횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## 03. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [13]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite_name = ['Men in Black (1997)' , 'Matrix, The (1999)' ,'Jurassic Park (1993)','Back to the Future (1985)','Shawshank Redemption, The (1994)']
my_favorite = [movies[movies['title'] == name]['movie_id'].values[0] for name in my_favorite_name]

# hong(맨끝에 추가되는 id)이라는 user_id가 위 영화를 5회씩 봤다고 가정하겠습니다.
hong = max(ratings['user_id']) + 1
my_playlist = pd.DataFrame({'user_id': [hong]*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':[hong]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,1580,5,NaN
1,6041,2571,5,NaN
2,6041,480,5,NaN
3,6041,1270,5,NaN
4,6041,318,5,NaN


## 04. CSR matrix를 직접 만들어 봅시다.

In [14]:
# 실습 위에 설명보고 이해해서 만들어보기

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 05. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [15]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 06. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [19]:
# 영화의 이름을 movie id로 바꿔주는 함수
def movie_name_to_id(name):
    return movies[movies['title']==name]['movie_id'].values[0]

In [20]:
my_vector, american_beauty_vector = als_model.user_factors[6041], als_model.item_factors[movie_name_to_id('American Beauty (1999)')]

In [21]:
# 나의 벡터
my_vector

array([-0.6280407 ,  0.2956025 , -0.06546354, -0.21902576,  0.02644725,
        0.01915325,  0.34031504,  0.40367484,  0.23169088, -0.06220978,
       -0.7014835 ,  1.0828938 , -0.8343029 , -0.6826763 , -0.11678414,
        0.42118615,  0.2094611 , -0.74561656,  0.08484034, -1.0395296 ,
        0.54352003, -0.76686966, -0.6329826 ,  1.0574021 , -0.06135674,
        0.2356638 ,  1.5866665 , -0.53876257,  0.34193027, -0.09849332,
        0.23559806, -0.72946286,  0.54398745, -0.5470282 , -0.13454711,
        0.03628593,  0.75712156,  0.09222657, -0.4249303 , -0.16137171,
        0.12614213,  0.19755308, -0.19723435,  0.3934108 ,  0.83695066,
        0.68691504, -0.43661812,  0.0815068 ,  0.7956882 ,  0.17555524,
        0.19456115, -0.1444794 ,  0.21272194,  0.2586847 ,  0.01321602,
       -1.330372  ,  0.3614211 , -0.61352074, -0.36250073,  0.06464734,
       -0.02402064,  0.2988261 , -0.05628537,  0.68098277,  0.11289663,
       -0.19153301, -0.81697315, -0.27511454, -0.13498175,  0.63

In [22]:
# american beauty 영화의 vector
american_beauty_vector

array([ 0.00210496,  0.02558163,  0.0272997 , -0.03247653, -0.03344492,
        0.00192331, -0.01899153,  0.00977926,  0.00586442,  0.04330087,
        0.01302161,  0.00892179, -0.03680665, -0.00517114,  0.00629263,
        0.0168648 ,  0.02172243, -0.04675321,  0.02478758,  0.0176519 ,
        0.02140031,  0.03196715,  0.01550685,  0.02528441,  0.04136225,
        0.01780271,  0.01496016, -0.02455216,  0.07020152, -0.007943  ,
        0.00147463, -0.00481511,  0.02916721, -0.02818749,  0.01951392,
       -0.00851053, -0.05012728,  0.01363944, -0.02033817, -0.01583074,
       -0.01012485,  0.00237804, -0.00302987,  0.02139857, -0.00136077,
        0.02695094, -0.00706135, -0.02899705, -0.00357107, -0.0209038 ,
        0.01004835,  0.04333567,  0.03659362,  0.01160991,  0.01368099,
        0.00334232,  0.03882225, -0.01182072, -0.00399048,  0.04522125,
       -0.00962777,  0.01988981,  0.01078079, -0.04494256,  0.00699424,
       -0.00076653,  0.014689  ,  0.01385592,  0.00803001,  0.03

In [23]:
# 영화 이름을 넣으면 추천정도를 알려주는 함수
def score_movie(movie_name):
    my_vector, movie_vector = als_model.user_factors[6041], als_model.item_factors[movie_name_to_id(movie_name)]
    return np.dot(my_vector, movie_vector)

In [24]:
for i in movie_top30.index:
    tmp = movies[movies['movie_id']==i]['title'].values[0]
    print(tmp," :",  score_movie(tmp))

American Beauty (1999)  : 0.27524
Star Wars: Episode IV - A New Hope (1977)  : 0.31358603
Star Wars: Episode V - The Empire Strikes Back (1980)  : 0.28482932
Star Wars: Episode VI - Return of the Jedi (1983)  : 0.26663762
Saving Private Ryan (1998)  : 0.35255527
Terminator 2: Judgment Day (1991)  : 0.66099983
Silence of the Lambs, The (1991)  : 0.42529732
Raiders of the Lost Ark (1981)  : 0.15779874
Back to the Future (1985)  : 0.5368912
Matrix, The (1999)  : 0.5938476
Jurassic Park (1993)  : 0.75297934
Sixth Sense, The (1999)  : 0.17088306
Fargo (1996)  : 0.29056627
Braveheart (1995)  : 0.5103554
Men in Black (1997)  : 0.66442907
Schindler's List (1993)  : 0.29884744
Princess Bride, The (1987)  : 0.20950533
Shakespeare in Love (1998)  : 0.059372175
L.A. Confidential (1997)  : 0.09070022
Shawshank Redemption, The (1994)  : 0.42111388
Godfather, The (1972)  : -0.001142198
Groundhog Day (1993)  : 0.15836723
E.T. the Extra-Terrestrial (1982)  : 0.20622323
Being John Malkovich (1999)  : -0

## 07. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [25]:
# Toystory와 비슷한 영화 추천
favorite_movie = 'Toy Story (1995)'
movie_id = movie_name_to_id(favorite_movie)
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1, 0.9999999),
 (3114, 0.80172867),
 (34, 0.58949995),
 (2355, 0.57473695),
 (588, 0.57394207),
 (1265, 0.51751137),
 (364, 0.4351031),
 (595, 0.4322702),
 (2321, 0.41524768),
 (1923, 0.39124677),
 (356, 0.3532873),
 (1566, 0.34571257),
 (2396, 0.34314036),
 (2078, 0.33026886),
 (2687, 0.32938576)]

In [26]:
for i, k in similar_movie:
    print(movies[movies['movie_id']==i]['title'].values[0],' :', k)

Toy Story (1995)  : 0.9999999
Toy Story 2 (1999)  : 0.80172867
Babe (1995)  : 0.58949995
Bug's Life, A (1998)  : 0.57473695
Aladdin (1992)  : 0.57394207
Groundhog Day (1993)  : 0.51751137
Lion King, The (1994)  : 0.4351031
Beauty and the Beast (1991)  : 0.4322702
Pleasantville (1998)  : 0.41524768
There's Something About Mary (1998)  : 0.39124677
Forrest Gump (1994)  : 0.3532873
Hercules (1997)  : 0.34571257
Shakespeare in Love (1998)  : 0.34314036
Jungle Book, The (1967)  : 0.33026886
Tarzan (1999)  : 0.32938576


In [27]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])][['title','genre']]

,title,genre
0,Toy Story (1995),Animation|Children's|Comedy
33,Babe (1995),Children's|Comedy|Drama
352,Forrest Gump (1994),Comedy|Romance|War
360,"Lion King, The (1994)",Animation|Children's|Musical
584,Aladdin (1992),Animation|Children's|Comedy|Musical
591,Beauty and the Beast (1991),Animation|Children's|Musical
1245,Groundhog Day (1993),Comedy|Romance
1526,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
1854,There's Something About Mary (1998),Comedy
2009,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical


## 08. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [28]:
# user 6041(hong)이 좋아할만한 영화를 추천
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.66099983),
 (110, 0.5103554),
 (593, 0.42529738),
 (2916, 0.4222901),
 (2028, 0.35255527),
 (260, 0.31358603),
 (527, 0.29884744),
 (608, 0.29056627),
 (1704, 0.28488272),
 (1196, 0.28482926),
 (780, 0.28116232),
 (2858, 0.27523997),
 (1193, 0.27268308),
 (1210, 0.26663762),
 (457, 0.2491868),
 (1573, 0.24221295),
 (1240, 0.2301282),
 (1527, 0.22514883),
 (2791, 0.21668965),
 (32, 0.21511093)]

In [32]:
# 좋아할만한 영화와 수치 장르 확인
for i, k in movie_recommended:
    print(movies[movies['movie_id']==i]['title'].values[0],' :',k)

Terminator 2: Judgment Day (1991)  : 0.66099983
Braveheart (1995)  : 0.5103554
Silence of the Lambs, The (1991)  : 0.42529738
Total Recall (1990)  : 0.4222901
Saving Private Ryan (1998)  : 0.35255527
Star Wars: Episode IV - A New Hope (1977)  : 0.31358603
Schindler's List (1993)  : 0.29884744
Fargo (1996)  : 0.29056627
Good Will Hunting (1997)  : 0.28488272
Star Wars: Episode V - The Empire Strikes Back (1980)  : 0.28482926
Independence Day (ID4) (1996)  : 0.28116232
American Beauty (1999)  : 0.27523997
One Flew Over the Cuckoo's Nest (1975)  : 0.27268308
Star Wars: Episode VI - Return of the Jedi (1983)  : 0.26663762
Fugitive, The (1993)  : 0.2491868
Face/Off (1997)  : 0.24221295
Terminator, The (1984)  : 0.2301282
Fifth Element, The (1997)  : 0.22514883
Airplane! (1980)  : 0.21668965
Twelve Monkeys (1995)  : 0.21511093


In [30]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])][['title','genre']]

,title,genre
31,Twelve Monkeys (1995),Drama|Sci-Fi
108,Braveheart (1995),Action|Drama|War
257,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
453,"Fugitive, The (1993)",Action|Thriller
523,Schindler's List (1993),Drama|War
585,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
589,"Silence of the Lambs, The (1991)",Drama|Thriller
604,Fargo (1996),Crime|Drama|Thriller
770,Independence Day (ID4) (1996),Action|Sci-Fi|War
1176,One Flew Over the Cuckoo's Nest (1975),Drama


## 09. 회고

1. factor 와 iteration
    - 학습시 두개의 값을 높이니 처음 선정해놓은 5개의 값들은 올라갔지만 그 외에는 점수가 낮아졌다.
    - 적절한 parameter를 정하는 것이 중요하다는 생각이 들엇다. 하지만 선호도라는 것이 사람의 마음이고 수치로 표현하는 것이 애매하다보니 어떻게 피드백을 받고 수정을 해가야할까 의문이 들었다.

2. Series, Pandas
    - Series와 Pandas 다루는 것은 아직도 낯설고 마음대로 다루지 못했다. 코딩테스트에서 준비했던 자료구조와 리스트와는 다르다. data를 어떻게 다룰지에대해서는 혼자서 해보고 많이 부딪혀봐야겠다.
    
3. 성능
    - 비슷한 영화와 내가 선택한 영화를 기반으로 재밌어 할만한 영화들을 추천해줬다. 추천해준 영화들을 보면 장르와 많은 연관이 있었다. 런타임시간, 시간대 등의 정보가 더 있으면 재밌는 결과가 나올 것이라는 생각이 들었다.